In [32]:
import pandas as pd

# Chargement du dataset final
df = pd.read_csv("accidents_finalV3.csv", sep=";", low_memory=False)

print("Shape :", df.shape)
print("Colonnes :", list(df.columns))

display(df.head(3))


Shape : (110124, 33)
Colonnes : ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lat', 'long', 'com', 'dep', 'lum', 'atm', 'agg', 'col', 'adr', 'catr', 'circ', 'surf', 'infra', 'situ', 'plan', 'prof', 'vma', 'nb_vehicules', 'has_2RM', 'has_VL', 'has_PL', 'has_TC', 'has_MD', 'has_OTHER', 'choc_multiple', 'nb_usagers', 'grav_max', 'age_moyen']


Num_Acc  jour  mois    an   hrmn            lat            long  \
0  202200000001    19    10  2022  16:15  44,5594200000    4,7257200000   
1  202200000002    20    10  2022  08:34  46,9258100000    6,3462000000   
2  202200000003    20    10  2022  17:15  48,4931620000   -2,7604390000   

     com dep  lum  ...  has_2RM  has_VL  has_PL has_TC  has_MD  has_OTHER  \
0  26198  26    1  ...        0       1       0      0       1          0   
1  25204  25    1  ...        0       1       0      0       0          0   
2  22360  22    1  ...        0       1       0      0       0          0   

   choc_multiple  nb_usagers  grav_max  age_moyen  
0              0           2         3       44.0  
1              0           2         4       43.0  
2              0           2         4       27.5  

[3 rows x 33 columns]

In [33]:
import numpy as np
import pandas as pd

# Comptage des NaN
nan_count = df.isna().sum()

# Comptage des -1 (sur colonnes numériques uniquement)
minus_one_count = (df == -1).sum(numeric_only=True)

summary = (
    pd.DataFrame({
        "nb_NaN": nan_count,
        "nb_-1": minus_one_count
    })
    .fillna(0)
    .astype(int)
    .sort_values(by=["nb_NaN", "nb_-1"], ascending=False)
)

summary



nb_NaN  nb_-1
adr              2622      0
vma              1910      0
circ                0   7046
infra               0   1200
col                 0     76
situ                0     63
prof                0     61
surf                0     60
plan                0     43
atm                 0      5
lum                 0      4
Num_Acc             0      0
jour                0      0
mois                0      0
an                  0      0
hrmn                0      0
lat                 0      0
long                0      0
com                 0      0
dep                 0      0
agg                 0      0
catr                0      0
nb_vehicules        0      0
has_2RM             0      0
has_VL              0      0
has_PL              0      0
has_TC              0      0
has_MD              0      0
has_OTHER           0      0
choc_multiple       0      0
nb_usagers          0      0
grav_max            0      0
age_moyen           0      0

In [34]:
import numpy as np

# ============================================================
# 1️⃣ Adresse : NaN → "inconnu"
# ============================================================

df["adr"] = df["adr"].fillna("inconnu")


# ============================================================
# 2️⃣ VMA : imputation conditionnelle par catégorie de route
# ============================================================

# médiane par catégorie de route
vma_by_catr = df.groupby("catr")["vma"].median()

# médiane globale (fallback)
vma_global = df["vma"].median()

def impute_vma(row):
    if pd.isna(row["vma"]):
        return vma_by_catr.get(row["catr"], vma_global)
    return row["vma"]

df["vma"] = df.apply(impute_vma, axis=1)

# Vérification
print("NaN restants dans adr :", df["adr"].isna().sum())
print("NaN restants dans vma :", df["vma"].isna().sum())


NaN restants dans adr : 0
NaN restants dans vma : 0


In [35]:
# ============================================================
# AFFICHAGE DES BORNES POUR PLUSIEURS QUANTILES
# ============================================================

quantile_sets = {
    "99%": (0.005, 0.995),
    "98%": (0.01, 0.99),
    "97%": (0.015, 0.985),
    "95%": (0.025, 0.975),
}

cols = ["vma", "age_moyen", "nb_vehicules", "nb_usagers"]

rows = []

for col in cols:
    for label, (q_low, q_high) in quantile_sets.items():
        low = df[col].quantile(q_low)
        high = df[col].quantile(q_high)
        rows.append({
            "variable": col,
            "quantiles": label,
            "q_low": q_low,
            "q_high": q_high,
            "borne_basse": round(low, 2),
            "borne_haute": round(high, 2),
        })

bounds_df = pd.DataFrame(rows)

bounds_df.sort_values(["variable", "q_low"])


variable quantiles  q_low  q_high  borne_basse  borne_haute
4      age_moyen       99%  0.005   0.995        13.00        84.00
5      age_moyen       98%  0.010   0.990        14.67        80.00
6      age_moyen       97%  0.015   0.985        15.50        77.00
7      age_moyen       95%  0.025   0.975        17.00        73.67
12    nb_usagers       99%  0.005   0.995         1.00         8.00
13    nb_usagers       98%  0.010   0.990         1.00         7.00
14    nb_usagers       97%  0.015   0.985         1.00         6.00
15    nb_usagers       95%  0.025   0.975         1.00         5.00
8   nb_vehicules       99%  0.005   0.995         1.00         4.00
9   nb_vehicules       98%  0.010   0.990         1.00         4.00
10  nb_vehicules       97%  0.015   0.985         1.00         3.00
11  nb_vehicules       95%  0.025   0.975         1.00         3.00
0            vma       99%  0.005   0.995        20.00       130.00
1            vma       98%  0.010   0.990        30.00       130.00
2            vma       97%  0.015   0.985        30.00       130.00
3            vma       95%  0.025   0.975        30.00       110.00

In [36]:
import pandas as pd
import numpy as np

# Variables et quantiles à tester
quantile_sets = {
    "99%": (0.005, 0.995),
    "98%": (0.01, 0.99),
    "97%": (0.015, 0.985),
    "95%": (0.025, 0.975),
}

cols = ["vma", "age_moyen", "nb_vehicules", "nb_usagers"]

rows = []

for col in cols:
    series = df[col].dropna()
    
    for label, (q_low, q_high) in quantile_sets.items():
        a = series.quantile(q_low)
        b = series.quantile(q_high)
        
        below = series[series < a]
        above = series[series > b]
        
        rows.append({
            "variable": col,
            "quantiles": label,
            "borne_basse": round(a, 2),
            "borne_haute": round(b, 2),
            "nb_valeurs_<a": below.shape[0],
            "median_<a": round(below.median(), 2) if not below.empty else np.nan,
            "nb_valeurs_>b": above.shape[0],
            "median_>b": round(above.median(), 2) if not above.empty else np.nan,
        })

diagnostic_bounds = pd.DataFrame(rows)

diagnostic_bounds.sort_values(
    by=["variable", "quantiles"],
    ascending=[True, False]
)


variable quantiles  borne_basse  borne_haute  nb_valeurs_<a  \
4      age_moyen       99%        13.00        84.00            442   
5      age_moyen       98%        14.67        80.00           1101   
6      age_moyen       97%        15.50        77.00           1555   
7      age_moyen       95%        17.00        73.67           2663   
12    nb_usagers       99%         1.00         8.00              0   
13    nb_usagers       98%         1.00         7.00              0   
14    nb_usagers       97%         1.00         6.00              0   
15    nb_usagers       95%         1.00         5.00              0   
8   nb_vehicules       99%         1.00         4.00              0   
9   nb_vehicules       98%         1.00         4.00              0   
10  nb_vehicules       97%         1.00         3.00              0   
11  nb_vehicules       95%         1.00         3.00              0   
0            vma       99%        20.00       130.00            244   
1            vma       98%        30.00       130.00            727   
2            vma       97%        30.00       130.00            727   
3            vma       95%        30.00       110.00            727   

    median_<a  nb_valeurs_>b  median_>b  
4        11.0            477       87.0  
5        13.0           1033       84.0  
6        14.0           1649       81.5  
7        15.0           2746       78.5  
12        NaN            379       10.0  
13        NaN            645        9.0  
14        NaN           1210        8.0  
15        NaN           2468        6.0  
8         NaN            424        5.0  
9         NaN            424        5.0  
10        NaN           1564        4.0  
11        NaN           1564        4.0  
0        10.0             14      500.0  
1        20.0             14      500.0  
2        20.0             14      500.0  
3        20.0           2216      130.0

In [37]:
"""
age_moyen — quantile 98 %

borne basse = 14.67

borne haute = 80.00

médiane < a = 13.0

médiane > b = 84.0

nb_usagers — quantile 97 %

borne haute = 6

médiane > b = 8.0

nb_vehicules — quantile 95 %

borne haute = 3

médiane > b = 4.0

vma — quantile 97 % (⚠️ règle métier)

borne basse = 30

borne haute = 130

PAS de médiane ici (borne réglementaire)
"""

'\nage_moyen — quantile 98 %\n\nborne basse = 14.67\n\nborne haute = 80.00\n\nmédiane < a = 13.0\n\nmédiane > b = 84.0\n\nnb_usagers — quantile 97 %\n\nborne haute = 6\n\nmédiane > b = 8.0\n\nnb_vehicules — quantile 95 %\n\nborne haute = 3\n\nmédiane > b = 4.0\n\nvma — quantile 97 % (⚠️ règle métier)\n\nborne basse = 30\n\nborne haute = 130\n\nPAS de médiane ici (borne réglementaire)\n'

In [38]:
# ============================================================
# CORRECTION DE vma > 130 PAR MOYENNE DU TYPE DE ROUTE
# ============================================================

# Calcul de la moyenne de vma par type de route (hors valeurs aberrantes)
vma_mean_by_catr = (
    df.loc[df["vma"].notna() & (df["vma"] <= 130)]
    .groupby("catr")["vma"]
    .mean()
)

print("Moyenne de vma par type de route (catr) :")
print(vma_mean_by_catr)

# Masque des valeurs aberrantes
mask_vma_aberrant = df["vma"].notna() & (df["vma"] > 130)

print("\nNombre de vma > 130 à corriger :", mask_vma_aberrant.sum())

# Remplacement conditionnel
df.loc[mask_vma_aberrant, "vma"] = (
    df.loc[mask_vma_aberrant, "catr"]
    .map(vma_mean_by_catr)
)

# Vérification finale
print("\nAprès correction vma :")
print("min :", df["vma"].min())
print("max :", df["vma"].max())

print("\nValeurs restantes vma > 130 :", (df["vma"] > 130).sum())


Moyenne de vma par type de route (catr) :
catr
1    98.334315
2    75.893935
3    63.218088
4    44.570823
5    39.098361
6    32.899254
7    55.733029
9    46.723711
Name: vma, dtype: float64

Nombre de vma > 130 à corriger : 14

Après correction vma :
min : 1.0
max : 130.0

Valeurs restantes vma > 130 : 0


In [39]:
# ============================================================
# CLIPPING INTELLIGENT – age_moyen (quantile 98 %)
# ============================================================

low_age = 14.67
high_age = 80.00
median_low_age = 13.0
median_high_age = 84.0

mask_low = df["age_moyen"].notna() & (df["age_moyen"] < low_age)
mask_high = df["age_moyen"].notna() & (df["age_moyen"] > high_age)

print("age_moyen < borne basse :", mask_low.sum())
print("age_moyen > borne haute :", mask_high.sum())

df.loc[mask_low, "age_moyen"] = median_low_age
df.loc[mask_high, "age_moyen"] = median_high_age

print("\nAprès correction age_moyen :")
print("min :", df["age_moyen"].min())
print("max :", df["age_moyen"].max())


age_moyen < borne basse : 1116
age_moyen > borne haute : 1033

Après correction age_moyen :
min : 13.0
max : 84.0


In [40]:
# ============================================================
# CLIPPING INTELLIGENT – nb_usagers (quantile 97 %)
# ============================================================

high_usagers = 6
median_high_usagers = 8.0

mask_high = df["nb_usagers"].notna() & (df["nb_usagers"] > high_usagers)

print("nb_usagers > borne haute :", mask_high.sum())

df.loc[mask_high, "nb_usagers"] = median_high_usagers

print("\nAprès correction nb_usagers :")
print("min :", df["nb_usagers"].min())
print("max :", df["nb_usagers"].max())


nb_usagers > borne haute : 1210

Après correction nb_usagers :
min : 1
max : 8


In [41]:
# ============================================================
# CLIPPING INTELLIGENT – nb_vehicules (quantile 95 %)
# ============================================================

high_veh = 3
median_high_veh = 4.0

mask_high = df["nb_vehicules"].notna() & (df["nb_vehicules"] > high_veh)

print("nb_vehicules > borne haute :", mask_high.sum())

df.loc[mask_high, "nb_vehicules"] = median_high_veh

print("\nAprès correction nb_vehicules :")
print("min :", df["nb_vehicules"].min())
print("max :", df["nb_vehicules"].max())


nb_vehicules > borne haute : 1564

Après correction nb_vehicules :
min : 1
max : 4


In [42]:
# ============================================================
# CORRECTION FINALE vma – borne basse métier uniquement
# ============================================================

low_vma = 30

mask_low = df["vma"].notna() & (df["vma"] < low_vma)

print("vma < borne basse (à corriger) :", mask_low.sum())

df.loc[mask_low, "vma"] = low_vma

print("\nAprès correction finale vma :")
print("min :", df["vma"].min())
print("max :", df["vma"].max())


vma < borne basse (à corriger) : 727

Après correction finale vma :
min : 30.0
max : 130.0


In [43]:
import numpy as np
import pandas as pd

# Comptage des NaN
nan_count = df.isna().sum()

# Comptage des -1 (sur colonnes numériques uniquement)
minus_one_count = (df == -1).sum(numeric_only=True)

summary = (
    pd.DataFrame({
        "nb_NaN": nan_count,
        "nb_-1": minus_one_count
    })
    .fillna(0)
    .astype(int)
    .sort_values(by=["nb_NaN", "nb_-1"], ascending=False)
)

summary


nb_NaN  nb_-1
circ                0   7046
infra               0   1200
col                 0     76
situ                0     63
prof                0     61
surf                0     60
plan                0     43
atm                 0      5
lum                 0      4
Num_Acc             0      0
jour                0      0
mois                0      0
an                  0      0
hrmn                0      0
lat                 0      0
long                0      0
com                 0      0
dep                 0      0
agg                 0      0
adr                 0      0
catr                0      0
vma                 0      0
nb_vehicules        0      0
has_2RM             0      0
has_VL              0      0
has_PL              0      0
has_TC              0      0
has_MD              0      0
has_OTHER           0      0
choc_multiple       0      0
nb_usagers          0      0
grav_max            0      0
age_moyen           0      0

In [44]:
# ============================================================
# TRANSFORMATION DE circ (régime de circulation)
# ============================================================

# Mapping BAAC
circ_map = {
    1: "sens_unique",
    2: "double_sens",
    3: "chaussees_separees",
    4: "voies_affectees"
}

# Remplacement des valeurs
df["circ"] = df["circ"].map(circ_map).fillna("inconnu")

# Passage en catégoriel
df["circ"] = df["circ"].astype("category")

# Vérification
print(df["circ"].value_counts())
print(df["circ"].dtype)


circ
double_sens           67967
sens_unique           19674
chaussees_separees    14822
inconnu                7046
voies_affectees         615
Name: count, dtype: int64
category


In [45]:
# ============================================================
# TRANSFORMATION DE infra (aménagements de l'infrastructure)
# ============================================================

infra_map = {
    0: "aucun_amenagement",
    1: "piste_cyclable",
    2: "voie_reservee",
    3: "bande_arret_urgence",
    4: "passage_pieton",
    5: "zone_30",
    6: "aire_pietonne",
    8: "autre"
}

df["infra"] = df["infra"].map(infra_map).fillna("inconnu")
df["infra"] = df["infra"].astype("category")

# Vérification
print(df["infra"].value_counts())
print(df["infra"].dtype)


infra
aucun_amenagement      92687
zone_30                 5605
inconnu                 5145
voie_reservee           1920
bande_arret_urgence     1286
piste_cyclable          1262
aire_pietonne           1109
autre                    807
passage_pieton           303
Name: count, dtype: int64
category


In [46]:
# ============================================================
# TRANSFORMATION DE col (type de collision)
# ============================================================

col_map = {
    1: "frontale",
    2: "arriere",
    3: "laterale",
    4: "chaine",
    5: "multiple",
    6: "autre"
}

df["col"] = df["col"].map(col_map).fillna("inconnu")
df["col"] = df["col"].astype("category")

# Vérification
print(df["col"].value_counts())
print(df["col"].dtype)




col
laterale    33214
autre       32664
arriere     14688
frontale    11806
inconnu     10805
chaine       3787
multiple     3160
Name: count, dtype: int64
category


In [47]:
# ============================================================
# TRANSFORMATION DE situ (situation de l'accident)
# ============================================================

situ_map = {
    1: "chaussee",
    2: "bande_arret_urgence",
    3: "accotement",
    4: "trottoir",
    5: "piste_cyclable",
    6: "terre_plein_central",
    8: "autre"
}

df["situ"] = df["situ"].map(situ_map).fillna("inconnu")
df["situ"] = df["situ"].astype("category")

# Vérification
print(df["situ"].value_counts())
print(df["situ"].dtype)


situ
chaussee               90446
accotement              8143
autre                   4001
piste_cyclable          2798
trottoir                2596
terre_plein_central     1362
bande_arret_urgence      715
inconnu                   63
Name: count, dtype: int64
category


In [48]:
# ============================================================
# TRANSFORMATION DE prof (profil longitudinal de la chaussée)
# ============================================================

prof_map = {
    1: "plat",
    2: "pente",
    3: "sommet_cote",
    4: "bas_cote"
}

df["prof"] = df["prof"].map(prof_map).fillna("inconnu")
df["prof"] = df["prof"].astype("category")

# Vérification
print(df["prof"].value_counts())
print(df["prof"].dtype)


prof
plat           89314
pente          17566
sommet_cote     1702
bas_cote        1481
inconnu           61
Name: count, dtype: int64
category


In [49]:
# ============================================================
# TRANSFORMATION DE surf (état de la chaussée - regroupé)
# ============================================================

surf_map = {
    1: "normale",
    2: "mouillee",
    3: "mouillee",
    4: "glissante",
    5: "glissante",
    6: "degradee",
    7: "degradee"
}

df["surf"] = df["surf"].map(surf_map).fillna("inconnu")
df["surf"] = df["surf"].astype("category")

# Vérification
print(df["surf"].value_counts())
print(df["surf"].dtype)


surf
normale      89217
mouillee     19528
inconnu        813
degradee       410
glissante      156
Name: count, dtype: int64
category


In [50]:
# ============================================================
# TRANSFORMATION DE plan (géométrie de la route - regroupée)
# ============================================================

plan_map = {
    1: "ligne_droite",
    2: "courbe",
    3: "courbe",
    4: "courbe_complexe"
}

df["plan"] = df["plan"].map(plan_map).fillna("inconnu")
df["plan"] = df["plan"].astype("category")

# Vérification
print(df["plan"].value_counts())
print(df["plan"].dtype)


plan
ligne_droite       88513
courbe             20077
courbe_complexe     1491
inconnu               43
Name: count, dtype: int64
category


In [51]:
# ============================================================
# TRAITEMENT ROBUSTE DE lum (conditions de luminosité)
# ============================================================

df["lum"] = pd.to_numeric(df["lum"], errors="coerce")

def infer_lum_from_hour(hrmn):
    try:
        hour = int(str(hrmn).split(":")[0])
        return 1 if 7 <= hour <= 19 else 4
    except:
        return 1

# Imputation des -1
mask_lum_missing = df["lum"] == -1
df.loc[mask_lum_missing, "lum"] = df.loc[mask_lum_missing, "hrmn"].apply(infer_lum_from_hour)

# Mapping
lum_map = {
    1: "plein_jour",
    2: "crepuscule_aube",
    3: "nuit_sans_eclairage",
    4: "nuit_avec_eclairage"
}

df["lum"] = df["lum"].map(lum_map).fillna("plein_jour")
df["lum"] = df["lum"].astype("category")

# Vérification
print(df["lum"].value_counts())
print(df["lum"].dtype)
print("NaN lum :", df["lum"].isna().sum())


lum
plein_jour             90190
nuit_sans_eclairage    11541
crepuscule_aube         7232
nuit_avec_eclairage     1161
Name: count, dtype: int64
category
NaN lum : 0


In [52]:
# ============================================================
# TRAITEMENT ROBUSTE DE atm (conditions atmosphériques)
# ============================================================

# On repart des codes numériques
df["atm"] = pd.to_numeric(df["atm"], errors="coerce")

# Imputation des -1 par la modalité majoritaire
atm_mode = df.loc[df["atm"] != -1, "atm"].mode()[0]
df.loc[df["atm"] == -1, "atm"] = atm_mode

# Mapping fonctionnel
atm_map = {
    1: "normales",
    2: "pluie",
    3: "pluie",
    4: "neige_grele",
    5: "brouillard_fumee",
    6: "vent_fort",
    7: "visibilite_degradee",
    8: "visibilite_degradee",
    9: "autre_meteo"
}

df["atm"] = df["atm"].map(atm_map).fillna("normales")
df["atm"] = df["atm"].astype("category")

# Vérification
print(df["atm"].value_counts())
print(df["atm"].dtype)
print("NaN atm :", df["atm"].isna().sum())


atm
normales               88021
pluie                  14050
visibilite_degradee     6257
brouillard_fumee         770
autre_meteo              471
vent_fort                308
neige_grele              247
Name: count, dtype: int64
category
NaN atm : 0


In [54]:
catr_map = {
    1: "autoroute",
    2: "route_nationale",
    3: "route_departementale",
    4: "voie_communale",
    5: "hors_reseau",
    6: "parking_ouvert",
    7: "voie_privee",
    9: "autre"
}

df["catr"] = df["catr"].map(catr_map).fillna("inconnu")
df["catr"] = df["catr"].astype("category")


In [55]:
# ============================================================
# TRANSFORMATION DE agg (agglomération)
# ============================================================

agg_map = {
    1: "hors_agglomeration",
    2: "en_agglomeration"
}

df["agg"] = df["agg"].map(agg_map).fillna("inconnu")
df["agg"] = df["agg"].astype("category")

# Vérification
print(df["agg"].value_counts())
print(df["agg"].dtype)


agg
en_agglomeration      69362
hors_agglomeration    40762
Name: count, dtype: int64
category


In [57]:
# ============================================================
# VÉRIFICATION FINALE : valeurs de la première ligne
# ============================================================

first_row = df.iloc[0]

for col in df.columns:
    print(f"{col:20s} : {first_row[col]} ({type(first_row[col]).__name__})")


Num_Acc              : 202200000001 (int64)
jour                 : 19 (int64)
mois                 : 10 (int64)
an                   : 2022 (int64)
hrmn                 : 16:15 (str)
lat                  : 44,5594200000 (str)
long                 : 4,7257200000 (str)
com                  : 26198 (str)
dep                  : 26 (str)
lum                  : plein_jour (str)
atm                  : normales (str)
agg                  : en_agglomeration (str)
col                  : laterale (str)
adr                  : TEIL(vieille route du) (str)
catr                 : voie_communale (str)
circ                 : double_sens (str)
surf                 : normale (str)
infra                : aucun_amenagement (str)
situ                 : chaussee (str)
plan                 : ligne_droite (str)
prof                 : plat (str)
vma                  : 50.0 (float64)
nb_vehicules         : 2 (int64)
has_2RM              : 0 (int64)
has_VL               : 1 (int64)
has_PL               : 0 (int64

In [58]:
import numpy as np
import pandas as pd

# Comptage des NaN
nan_count = df.isna().sum()

# Comptage des -1 (sur colonnes numériques uniquement)
minus_one_count = (df == -1).sum(numeric_only=True)

summary = (
    pd.DataFrame({
        "nb_NaN": nan_count,
        "nb_-1": minus_one_count
    })
    .fillna(0)
    .astype(int)
    .sort_values(by=["nb_NaN", "nb_-1"], ascending=False)
)

summary


nb_NaN  nb_-1
Num_Acc             0      0
jour                0      0
mois                0      0
an                  0      0
hrmn                0      0
lat                 0      0
long                0      0
com                 0      0
dep                 0      0
lum                 0      0
atm                 0      0
agg                 0      0
col                 0      0
adr                 0      0
catr                0      0
circ                0      0
surf                0      0
infra               0      0
situ                0      0
plan                0      0
prof                0      0
vma                 0      0
nb_vehicules        0      0
has_2RM             0      0
has_VL              0      0
has_PL              0      0
has_TC              0      0
has_MD              0      0
has_OTHER           0      0
choc_multiple       0      0
nb_usagers          0      0
grav_max            0      0
age_moyen           0      0

In [59]:
# ============================================================
# SAUVEGARDE DU DATASET CATÉGORIEL
# ============================================================

output_path = "./accident_pretraiter.csv"

df.to_csv(
    output_path,
    index=False,
    sep=';',
    encoding='utf-8'
)

print(f"✅ Dataset sauvegardé : {output_path}")
print(f"Nombre de lignes : {df.shape[0]}")
print(f"Nombre de colonnes : {df.shape[1]}")


✅ Dataset sauvegardé : ./accident_pretraiter.csv
Nombre de lignes : 110124
Nombre de colonnes : 33
